<a href="https://colab.research.google.com/github/entc-17-fyp-20/CM_Data_Visualizing/blob/main/Visualizing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Intro**

## Intro - Importing libraries

In [1]:
# !pip install -q dash
# !pip install  jupyter-dash
# !pip install -q dash_core_components
# !pip install -q dash_html_components
# !pip install -q dash_table
# !pip install pyngrok --quiet
# !pip install -U dataprep 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.random import seed

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [3]:
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
# from dataprep.eda import create_report

## Intro - Importing from Google Drive
[Drive Link](https://drive.google.com/drive/folders/123Lba0BhxQA2O6Beekbeq52gMr_9EZ5Z?usp=sharing)



In [97]:
#@title Select Dataset New { display-mode: "form" }
turbine = 'T01' #@param ["T01", "T02","T03","T22","T34","T54"]
path = '/content/drive/MyDrive/IIoT - Wind Turbine/FYP/Dataset/Merged/' + turbine + '.csv'
df1 = pd.read_csv(path)

In [98]:
#@title Time range { display-mode: "form" }
turbine = '9' #@param ["1", "2","3","4","5","6","7","8","9"]

df1 = df1.set_index(df1['Date_Time'])
if turbine =="1":
  df1 = df1[:'2017-03-19 05:10:00']
elif turbine =="2":
  df1 = df1['2017-04-13 16:10:00':"2017-05-31 00:00:00"]
elif turbine =="3":
  df1 = df1["2017-06-01 00:00:00":"2017-07-20 02:50:00"]
elif turbine =="4":
  df1 = df1['2017-07-21 08:10:00':'2017-08-14 05:20:00']
elif turbine =="5":
  df1 = df1['2018-01-15 12:00:00':"2018-02-10 23:50:00"]
elif turbine =="6":
  df1 = df1['2018-02-13 00:00:00':'2018-04-21 06:40:00']
elif turbine =="7":
  df1 = df1['2018-05-01 10:10:00':"2018-05-31 00:00:00"]
elif turbine =="8":
  df1 = df1['2018-06-01 00:00:00':'2018-08-14 06:50:00']
elif turbine =="9":
  df1 = df1['2018-08-21 07:00:00':]

## Intro - Date and Time

*   Adding "Hour" column

In [5]:
df1['Date_Time'] = pd.to_datetime(df1['Date_Time'], format="%Y-%m-%d %H:%M:%S")

def hr_func(ts):
    return ts.hour
    
df1["Hour"] = pd.to_numeric(df1['Date_Time'].apply(hr_func), downcast="float")

## Intro - Proprocessing
Data clensing

In [6]:
# df1 = df1[df1['Generator_RPM']>0] # Removing rows with zero Generator_RPM
# df1 = df1[df1['Gear_Oil_Tempeature']>0] # Removing negative temperature values
#df1 

# **DataPrep**
Report of data

In [ ]:
#@title Show Report { display-mode: "form" }
report = create_report(df1)
report.show()
report.save('Report')

# **Time series**

In [99]:
#@title One variable with time { display-mode: "form" }
column_name = "Gear_Bearing_Temperature" #@param ["Wind_Speed", "Ambient_Temperature", "Date_Time", "Gear_Bearing_Temperature", "Gear_Oil_Tempeature", "Generator_Bearing_Tempeature", "Hydrulic_Oil_Pressure", "Active_Power", "Wind_Speed", "Ambient_Temperature", "Hydrulic_Oil_Temperature", "Generator_RPM", "voltage1", "voltage2", "voltage3"]

fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df1['Date_Time']), y=list(df1[column_name]), name=column_name, yaxis="y"))

fig.add_trace(
    go.Scatter(x=list(df1['Date_Time']), y=[df1[column_name].mean()] * len(df1[column_name]), name="Average of "+ column_name, yaxis='y'))

# Set title
fig.update_layout(title_text= column_name)

#fig = px.line(df1, x="Date_Time" , y=column_name) 
fig.show()

In [ ]:
#@title Outlier Detection
#{ display-mode: "form" }
column_name = 'Power_act' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour', 'Power_act']
column_name1 = 'Active_Power' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']

gk = df1.groupby('Power_cate')
gk1 = gk.get_group('replaced')
gk2 = gk.get_group('actual')

fig = go.Figure()
fig.add_trace(go.Scatter(x=gk1["Date_Time"], y=gk1[column_name], mode='markers',
                  name='Replaced', yaxis="y"))
fig.add_trace(go.Scatter(x=gk2["Date_Time"], y=gk2[column_name], mode='markers',
                  name='Actual', yaxis="y"))
fig.add_trace(go.Scatter(x=df1["Date_Time"], y=df1[column_name1], mode='markers',
                  name='Actual', yaxis="y2"))

# Set title
fig.update_layout(
    title_text= column_name1 + " Outliers"
)

# Add range slider
fig.update_layout(
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.48],
        linecolor="#673ab7",
        mirror=True,
        #range=[0, 1000],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        title="kW",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.52, 1],
        linecolor="#795548",
        mirror=True,
        #range = [0,25],
        showline=True,
        side="left",
        tickfont={"color": "#795548"},
        tickmode="auto",
        ticks="",
        title="kW",
        titlefont={"color": "#795548"},
        type="linear",
        zeroline=False
    )
)

# Update layout
fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)

fig.show()

In [ ]:
#@title Three variables in one figure { display-mode: "form" }
column_name = 'Nacelle Temperature' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
column_name1 = 'Active_Power' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
column_name2 = 'Generator_RPM' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']

fig = go.Figure()
fig.add_trace(go.Scatter(x=df1["Date_Time"], y=df1[column_name],
                    mode='lines',
                    name='R'))
fig.add_trace(go.Scatter(x=df1["Date_Time"], y=df1[column_name1],
                    mode='lines',
                    name='S'))
fig.add_trace(go.Scatter(x=df1["Date_Time"], y=df1[column_name2],
                    mode='lines', name='T'))

fig.show()

In [42]:
#@title Three variables in three figures { display-mode: "form" }
column_name = 'Active_Power'#@param ["Wind_Speed", "Ambient_Temperature", "Date_Time", "Gear_Bearing_Temperature", "Gear_Oil_Tempeature", "Generator_Bearing_Tempeature", "Hydrulic_Oil_Pressure", "Active_Power", "Wind_Speed", "Ambient_Temperature", "Hydrulic_Oil_Temperature", "Generator_RPM", "voltage1", "voltage2", "voltage3"]
column_name1 = 'Wind_Speed' #@param ["Wind_Speed", "Ambient_Temperature", "Date_Time", "Gear_Bearing_Temperature", "Gear_Oil_Tempeature", "Generator_Bearing_Tempeature", "Hydrulic_Oil_Pressure", "Active_Power", "Wind_Speed", "Ambient_Temperature", "Hydrulic_Oil_Temperature", "Generator_RPM", "voltage1", "voltage2", "voltage3"]
column_name2 = 'Gear_Bearing_Temperature' #@param ["Wind_Speed", "Ambient_Temperature", "Date_Time", "Gear_Bearing_Temperature", "Gear_Oil_Tempeature", "Generator_Bearing_Tempeature", "Hydrulic_Oil_Pressure", "Active_Power", "Wind_Speed", "Ambient_Temperature", "Hydrulic_Oil_Temperature", "Generator_RPM", "voltage1", "voltage2", "voltage3"]

unit1 = column_name.split()[-1]
unit2 = column_name1.split()[-1]
unit3 = column_name2.split()[-1]

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df1['Date_Time']), y=list(df1[column_name]), name=column_name, yaxis="y"))

fig.add_trace(
    go.Scatter(x=list(df1['Date_Time']), y=list(df1[column_name1]), name=column_name1, yaxis="y2"))

fig.add_trace(
    go.Scatter(x=list(df1['Date_Time']), y=list(df1[column_name2]), name=column_name2, yaxis="y3"))

# Set title
fig.update_layout(
    title_text= column_name + " vs "+ column_name1 +" vs "+ column_name2
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=30,
                     label="30m",
                     step="minute",
                     stepmode="backward"),
                dict(count=1,
                     label="1h",
                     step="hour",
                     stepmode="backward"),
                dict(count=6,
                     label="6h",
                     step="hour",
                     stepmode="backward"),
                dict(count=1,
                     label="1d",
                     step="day",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.32],
        linecolor="#673ab7",
        mirror=True,
        range=[0, 1000],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        title=unit1,
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0.33, 0.66],
        linecolor="#795548",
        mirror=True,
        range = [0,25],
        showline=True,
        side="left",
        tickfont={"color": "#795548"},
        tickmode="auto",
        ticks="",
        title=unit2,
        titlefont={"color": "#795548"},
        type="linear",
        zeroline=False
    ),
     yaxis3=dict(
        anchor="x",
        autorange=True,
        domain=[0.67, 1],
        linecolor="#795548",
        mirror=True,
        range = [0,25],
        showline=True,
        side="left",
        tickfont={"color": "#795548"},
        tickmode="auto",
        ticks="",
        title=unit3,
        titlefont={"color": "#795548"},
        type="linear",
        zeroline=False
    )
)

# Update layout
fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)

fig.show()


# Stat

In [ ]:
#@title Box plot { display-mode: "form" }
column_name = 'Active_Power' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
print("Avg: ", end=" ")
print(df1[column_name].mean())
fig = px.box(df1, y=column_name)
fig.show()
#@markdown Upper fence = Q3 + (1.5 * IQR) |
#@markdown Lower fence = Q1 – (1.5 * IQR) | Inter interquartile range


In [ ]:
#@title 2D Scatter Plot { display-mode: "form" }
column_name = "Wind_Speed" #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
column_name1 = "Active_Power" #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour', 'Power_act']

fig = px.scatter(df1, x=column_name, y=column_name1)
fig.update_xaxes(ticks="inside")
fig.update_yaxes(ticks="inside", col=1)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, zerolinecolor='LightPink')
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, zerolinecolor='LightPink')
if column_name1 == "Power_act":
  fig.update_xaxes(range=[-1, 20])
  fig.update_yaxes(range=[-50, 900])
fig.show()

In [ ]:
#@title 3D Scatter Plot { display-mode: "form" }
column_name = "Wind_Speed" #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
column_name1 = "Date_Time" #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
column_name2 = "Active_Power" #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour', "Power_act"]

fig = px.scatter_3d(df1, x=column_name, y=column_name1, z=column_name2)
fig.update_xaxes(ticks="inside")
fig.update_yaxes(ticks="inside", col=1)
fig.update_layout(height=800)
fig.show()

In [ ]:
#@title Histogram { display-mode: "form" }
column_name = 'Active_Power' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
print("Avg: ", end=" ")
print(df1[column_name].mean())
fig = px.histogram(df1, x=column_name, nbins=20, histnorm='probability density')
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
#@title Distplot with normal curve { display-mode: "form" }
column_name = 'Wind_Speed' #@param ['Date_Time', 'Hydrulic_Oil_Pressure', 'Nacelle Temperature','Active_Power', 'Wind_Speed', 'Ambient_Temperature','Hydrulic_Oil_Temperature', 'Generator_RPM', 'voltage','Gearbox & Bearing temp', 'Hour']
fig = px.histogram(df1, x=column_name, nbins=20, histnorm='probability density')

group_labels = ['Normal Curve']
df2 = [df1[column_name]]
print("Avg: ", end=" ")
print(df1[column_name].mean())
fig = ff.create_distplot(df2, group_labels, bin_size=.8,
                         curve_type='normal')

fig.show()

In [ ]:
#@title Scatter Matrix { display-mode: "form" }
df2 = df1.drop(["Date_Time", "Hour"] ,axis=1)
fig = px.scatter_matrix(df2)
fig.show()

In [ ]:
#@title Correlation Heatmap { display-mode: "form" }
df2 = df1.drop(["Date_Time", "Hour"] ,axis=1)
# fig = px.imshow(df2.corr())
# fig.show()
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(df2.corr())

# Dash


In [ ]:
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN = '1wZCkywjJChLrQjiYfFJqFTZenq_2xGVJJ64eDVA29x9A2Nam'
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.connect(8050)

Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok
Opening tunnel named: http-8050-87138139-8ae6-41a0-b08f-dbac3509f450
t=2021-08-24T07:20:53+0000 lvl=info msg="no configuration paths supplied"
t=2021-08-24T07:20:53+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
t=2021-08-24T07:20:53+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
t=2021-08-24T07:20:53+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
t=2021-08-24T07:20:53+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2021-08-24T07:20:53+0000 lvl=info msg="client session established" obj=csess id=b2c2f5ed675e
t=2021-08-24T07:20:53+0000 lvl=info msg=start pg=/api/tunnels id=9d4bb8d5038830a0
t=2021-08-24T07:20:53+0000 lvl=info msg=end pg=/api/tunnels id=9d4bb8d5038830a0 status=200 dur=414.842µs
t=2021-08-24T07:20:53+0000 lvl=info msg=start pg=/api/tunnels id=85bda

<NgrokTunnel: "http://7b66-34-125-156-210.ngrok.io" -> "http://localhost:8050">

t=2021-08-24T07:20:53+0000 lvl=info msg=start pg="/api/tunnels/http-8050-87138139-8ae6-41a0-b08f-dbac3509f450 (http)" id=355ec4e2cfd2ec2c
t=2021-08-24T07:20:53+0000 lvl=info msg=end pg="/api/tunnels/http-8050-87138139-8ae6-41a0-b08f-dbac3509f450 (http)" id=355ec4e2cfd2ec2c status=200 dur=129.778µs


## Time series visualizing - non interactive

In [ ]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


t=2021-08-24T07:21:24+0000 lvl=info msg="join connections" obj=join id=ae5b3b1a437c l=127.0.0.1:8050 r=112.135.33.128:64745
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=9ca0ae6a4bf8 l=127.0.0.1:8050 r=112.135.33.128:65141
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=72a4bbd7f35d l=127.0.0.1:8050 r=112.135.33.128:65157
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=b586d6f67953 l=127.0.0.1:8050 r=112.135.33.128:64745
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=7eeca9454744 l=127.0.0.1:8050 r=112.135.33.128:64512
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=f97f1db33c0f l=127.0.0.1:8050 r=112.135.33.128:63601
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=59623e08afc3 l=127.0.0.1:8050 r=112.135.33.128:65008
t=2021-08-24T07:21:25+0000 lvl=info msg="join connections" obj=join id=f36bf8bd1dbf l=127.0.0.1:8050 r=112.135.33.128:63601
t=2021-0

## Scatter plots -  comparing correlation

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(children='Scatter Graphs'),  

    html.Div([html.Div([dcc.Dropdown(id='xaxis-column', options=[{'label': i, 'value': i} for i in df1.columns], value='Wind_Speed')], 
                       style={'width': '48%', 'display': 'inline-block'}),
              html.Div([
                      dcc.Dropdown(
                          id='yaxis-column',
                          options=[{'label': i, 'value': i} for i in df1.columns],
                          value='Active_Power'
                      )
                  ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='hour--slider',
        min=df1['Hour'].min(),
        max=df1['Hour'].max(),
        value=df1['Hour'].max(),
        marks={str(hour): str(hour) for hour in df1['Hour'].unique()},
        step=None
    ),
    html.Div(id='slider-output-container')
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    dash.dependencies.Output('slider-output-container', 'children'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('hour--slider','value'))
def update_graph(xaxis_column_name, yaxis_column_name,hour_slider=0): 
    dff = df1[df1['Hour'] == hour_slider]
    fig = px.scatter(dff, x=xaxis_column_name, y=yaxis_column_name, hover_name=dff['Date_Time'])
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    fig.update_xaxes(range=[df1[xaxis_column_name].min(), df1[xaxis_column_name].max()])
    fig.update_yaxes(range=[df1[yaxis_column_name].min(), df1[yaxis_column_name].max()])
    fig.update_layout(transition_duration=500)
    return fig, hour_slider

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

## Box plots -  outliers

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(children='Box plots'),               
    
    html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in df1.columns],
                value='Active_Power'
            )
        ]),
    dcc.Graph(id='indicator-graphic')
])

@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('yaxis-column', 'value'))
def update_graph(yaxis_column_name): 
    fig = px.box(df1, y=yaxis_column_name, hover_data=["Date_Time"]) # points="all" -  to get points
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 15, 'r': 0}, hovermode='closest')
    fig.update_layout(transition_duration=500)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)